# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import gkey


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cityWeatherDF = pd.read_csv("../WeatherPy/Data/CityWeatherData.csv")
cityWeatherDF.head()

,Unnamed: 0,City,Country,Cloudiness,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,bengkulu,Indonesia,78,1579747575,65,-3.80,102.27,85.17,3.33
1,1,broken hill,Australia,40,1579747575,27,-31.95,141.43,78.80,18.34
2,2,ellisras,South Africa,0,1579747575,70,-23.66,27.74,69.31,9.60
3,3,ushuaia,Argentina,75,1579747469,76,-54.80,-68.30,48.20,8.05
4,4,hobart,Australia,40,1579747576,59,-42.88,147.33,64.40,10.29


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# configure 
gmaps.configure(api_key=gkey)

# get parts for heatmap
humidity = [float(h) for h in cityWeatherDF['Humidity']]
locations = cityWeatherDF[['Lat','Lng']]
locations = locations.rename(columns={'Lat':'Latitude','Lng':'Longitude'})

# add heatmap
fig = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                    dissipating=False, max_intensity=max(humidity),
                                    point_radius=2)
fig.add_layer(heatmap_layer)
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# subset to only good weather
goodWeatherDF = cityWeatherDF[(cityWeatherDF['Max Temp'] > 65) & (cityWeatherDF['Max Temp'] < 80) & (cityWeatherDF['Humidity'] < 60) & (cityWeatherDF['Wind Speed'] < 8)]

goodWeatherDF = goodWeatherDF.reset_index()
goodWeatherDF

,index,Unnamed: 0,City,Country,Cloudiness,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,70,70,puerto ayora,Ecuador,27,1579747656,53,-0.74,-90.35,75.99,4.00
1,123,123,juba,Sudan,96,1579747445,20,4.85,31.58,68.68,3.69
2,198,198,kitgum,Uganda,46,1579747789,35,3.28,32.89,67.75,1.97
3,234,234,kerouane,Guinea,81,1579747839,44,9.27,-9.02,74.79,2.53
4,330,330,catio,Guinea-Bissau,98,1579747881,41,11.28,-15.25,73.83,2.04
5,476,476,homnabad,India,77,1579747978,56,17.77,77.13,70.27,6.40
6,521,521,gambela,Ethiopia,75,1579747988,25,8.25,34.58,69.64,1.57
7,531,531,san rafael,"Bolivia, Plurinational State of",0,1579747990,30,-34.61,-68.33,79.77,6.73


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
# make hotel dataframe
hotel_df = goodWeatherDF
# make basic api url and params
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {"location": "1,1",
          "rankby": "distance",
          "type": "hotel",
          "key": gkey}

In [6]:
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # get location from df
    loc = f"{float(row['Lat'])},{float(row['Lng'])}"

    # add keyword to params dict
    params['location'] = loc

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        hotel_df.loc[index, 'address'] = results[0]['vicinity']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 0: puerto ayora.
Closest hotel is El Mirador de Los Tuneles.
------------
Retrieving Results for Index 1: juba.
Closest hotel is Compound of China Overseas Engineering Group Co.,Ltd.
------------
Retrieving Results for Index 2: kitgum.
Closest hotel is Bardege Seventh Day Adventist Church.
------------
Retrieving Results for Index 3: kerouane.
Closest hotel is College A S T De Kéroua.
------------
Retrieving Results for Index 4: catio.
Closest hotel is AGUIPRODES Associação Guineense para Promoção do Desenvolvimento.
------------
Retrieving Results for Index 5: homnabad.
Closest hotel is S.P M.Ed College.
------------
Retrieving Results for Index 6: gambela.
Closest hotel is Makdela hotel resort.
------------
Retrieving Results for Index 7: san rafael.
Closest hotel is Lavandería Bubbles.
------------


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in goodWeatherDF.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [8]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# Display Map
fig

Figure(layout=FigureLayout(height='420px'))